# 如果报错就多restart几次！code没错，都是玄学

# 投资策略-step4

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [2]:
initial_investment = 1000000


In [3]:
stocklist_mag = ['NFLX', 'IDR', 'AEM', 'KGC', 'IAG', 'HMY', 'RCL', 'KMI', 'PHM', 'ADSK', 'NVDA', 'RL', 'RGLD', 'DHI','ORLA','ORCL','AUST','TMUS','FSM','NGD','EXPE','NEM','PLTR']

n_portfolios = 300000

end_date = datetime.today()
one_year_ago = end_date - timedelta(days=365)

# Download the risk-free rate (T-bill rate from ^IRX)
irx_data = yf.download('^IRX', start=one_year_ago, end=end_date)['Adj Close']
# Convert ^IRX to daily risk-free rate (as IRX is annualized, divide by 252)
daily_risk_free_rate = irx_data.mean() / 100 / 252  # Convert percentage to decimal and divide by 252 trading days

risk_free_rate = daily_risk_free_rate


[*********************100%***********************]  1 of 1 completed


In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Define the date range
end_date = datetime.today()
start_date = end_date - timedelta(days=2*365)

# Fetch stock data from Yahoo Finance
data = yf.download(stocklist_mag, start=start_date, end=end_date)['Adj Close']

# Calculate daily returns
returns = data.pct_change().dropna()

# Get market data for DJIA (used for calculating Treynor ratio and Jensen's Alpha)
market_data = yf.download("^DJI", start=start_date, end=end_date)['Adj Close']
market_returns = market_data.pct_change().dropna()


# Monte Carlo simulation parameters
n_assets = len(stocklist_mag)
results = np.zeros((4, n_portfolios))

# Store weights
all_weights = np.zeros((n_portfolios, n_assets))

# Monte Carlo simulation
for i in range(n_portfolios):
    # Generate random weights
    weights = np.random.random(n_assets)
    weights /= np.sum(weights)
    
    # Save weights
    all_weights[i, :] = weights
    
    # Portfolio returns and variance
    portfolio_return = np.sum(weights * returns.mean()) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
    portfolio_var = portfolio_std ** 2
    sharpe_ratio = (portfolio_return - risk_free_rate * 252) / portfolio_std

    results[0, i] = portfolio_return
    results[1, i] = portfolio_std
    results[2, i] = sharpe_ratio
    results[3, i] = portfolio_var

# Convert to DataFrame
results_frame = pd.DataFrame(results.T, columns=['Return', 'StdDev', 'SharpeRatio', 'Variance'])

# Find max Sharpe ratio portfolio
max_sharpe_idx = results_frame['SharpeRatio'].idxmax()
max_sharpe_portfolio = all_weights[max_sharpe_idx, :]

# Max return
max_return_idx = results_frame['Return'].idxmax()
max_return_portfolio = all_weights[max_return_idx, :]

# Min variance
min_variance_idx = results_frame['Variance'].idxmin()
min_variance_portfolio = all_weights[min_variance_idx, :]

# Treynor and Jensen's Alpha (use for optimized portfolios only)
def treynor_ratio(returns, weights, beta, market_return, risk_free_rate):
    portfolio_return = np.sum(weights * returns.mean()) * 252
    treynor = (portfolio_return - risk_free_rate * 252) / beta
    return treynor

def jensens_alpha(returns, weights, beta, market_return, risk_free_rate):
    portfolio_return = np.sum(weights * returns.mean()) * 252
    alpha = portfolio_return - (risk_free_rate * 252 + beta * (market_return.mean() - risk_free_rate * 252))
    return alpha

# Calculate portfolio betas
beta_values = np.zeros(n_portfolios)
market_excess_return = market_returns.mean() - risk_free_rate * 252

for i in range(n_portfolios):
    portfolio_beta = np.dot(all_weights[i], np.cov(returns.T, market_returns.T)[:-1, -1]) / market_returns.var()
    beta_values[i] = portfolio_beta

# Max Treynor Ratio
treynor_ratios = np.array([treynor_ratio(returns, all_weights[i], beta_values[i], market_returns, risk_free_rate)
                           for i in range(n_portfolios)])
max_treynor_idx = np.argmax(treynor_ratios)
max_treynor_portfolio = all_weights[max_treynor_idx, :]

# Max Jensen's Alpha
jensen_alphas = np.array([jensens_alpha(returns, all_weights[i], beta_values[i], market_returns, risk_free_rate)
                          for i in range(n_portfolios)])
max_jensen_alpha_idx = np.argmax(jensen_alphas)
max_jensen_alpha_portfolio = all_weights[max_jensen_alpha_idx, :]

# Display Results
portfolios = {
    'Max Sharpe Ratio': max_sharpe_portfolio,
    'Max Treynor Ratio': max_treynor_portfolio,
    'Max Jensen Alpha': max_jensen_alpha_portfolio,
    'Max Return': max_return_portfolio,
    'Min Variance': min_variance_portfolio
}

for name, portfolio in portfolios.items():
    portfolio_return = np.sum(portfolio * returns.mean()) * 252
    portfolio_std = np.sqrt(np.dot(portfolio.T, np.dot(returns.cov() * 252, portfolio)))
    print(f"\n{name} Portfolio:")
    print(f"Return: {portfolio_return}")
    print(f"Std Dev: {portfolio_std}")
    print(f"Weights: {portfolio}")


[*********************100%***********************]  23 of 23 completed
[*********************100%***********************]  1 of 1 completed


KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize

end_date = datetime.today()
one_year_ago = end_date - timedelta(days=2* 365)
backtest_data = data


# Step 2: Define the portfolio value calculation function
def portfolio_value(weights, stock_data, initial_investment):
    investment_per_stock = initial_investment * weights
    shares = investment_per_stock / stock_data.iloc[0]
    portfolio_value = (stock_data * shares).sum(axis=1)
    return portfolio_value

# Step 3: Implement Portfolio 1 (Monte Carlo Simulation with combined weights)
weights_combined = {
    'Max Return': 0.20,
    'Min Variance': 0.30,
    'Max Sharpe Ratio': 0.20,
    'Max Treynor Ratio': 0.15,
    'Max Jensen Alpha': 0.15
}

portfolios = {
    'Max Return': np.random.dirichlet(np.ones(len(stocklist_mag))),
    'Min Variance': np.random.dirichlet(np.ones(len(stocklist_mag))),
    'Max Sharpe Ratio': np.random.dirichlet(np.ones(len(stocklist_mag))),
    'Max Treynor Ratio': np.random.dirichlet(np.ones(len(stocklist_mag))),
    'Max Jensen Alpha': np.random.dirichlet(np.ones(len(stocklist_mag)))
}

combined_weights = np.zeros(len(stocklist_mag))
for name, weight in weights_combined.items():
    combined_weights += portfolios[name] * weight
combined_weights /= np.sum(combined_weights)

combined_portfolio_value = portfolio_value(combined_weights, backtest_data, initial_investment)

# Step 4: Implement Portfolio 2 (Equal weighting of three professional optimizations)
optimized_portfolio_values = {}

def portfolio_performance(weights, returns):
    portfolio_return = np.sum(weights * returns.mean()) * 252
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
    return portfolio_return, portfolio_volatility

# Define bounds to allow zero weights but not negative values (0 to 1)
bounds = tuple((0, 1) for asset in range(returns.shape[1]))

# Mean-Variance Optimization
def min_variance_portfolio(returns):
    num_assets = returns.shape[1]
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

    def portfolio_volatility(weights, returns):
        return portfolio_performance(weights, returns)[1]

    result = minimize(portfolio_volatility, num_assets * [1. / num_assets,], args=(returns,),
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

# Risk Parity Optimization
def risk_parity_portfolio(returns):
    cov_matrix = returns.cov()

    def risk_budget_objective(weights):
        total_portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
        marginal_risk_contribution = np.dot(cov_matrix, weights)
        risk_contribution = weights * marginal_risk_contribution / total_portfolio_variance
        return np.sum((risk_contribution - 1.0 / len(weights)) ** 2)

    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

    result = minimize(risk_budget_objective, np.ones(returns.shape[1]) / returns.shape[1],
                      bounds=bounds, constraints=constraints)
    return result.x

# Max Sharpe Ratio Optimization
def max_sharpe_ratio_portfolio(returns, risk_free_rate):
    num_assets = returns.shape[1]
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

    def negative_sharpe_ratio(weights, returns, risk_free_rate):
        portfolio_return, portfolio_volatility = portfolio_performance(weights, returns)
        return -(portfolio_return - risk_free_rate * 252) / portfolio_volatility

    result = minimize(negative_sharpe_ratio, num_assets * [1. / num_assets,], args=(returns, risk_free_rate),
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x


# Generate weights for each optimization strategy
mean_variance_weights = min_variance_portfolio(returns)
risk_parity_weights = risk_parity_portfolio(returns)
max_sharpe_weights = max_sharpe_ratio_portfolio(returns, daily_risk_free_rate)

# Combine the three optimized portfolios equally
combined_optimized_weights = (mean_variance_weights + risk_parity_weights + max_sharpe_weights) / 3

# Backtest Portfolio 2 (Combined Professional Optimized)
optimized_portfolio_values['Combined Professional'] = portfolio_value(combined_optimized_weights, backtest_data, initial_investment)

# Step 6: Fetch major indexes for comparison
index_symbols = ['^GSPC', '^DJI', '^IXIC']
index_data = yf.download(index_symbols, start=one_year_ago, end=end_date)['Adj Close']

# Step 7: Plot the comparison of all portfolios and major indexes
def normalize(values):
    return values / values.iloc[0]

normalized_combined_portfolio = normalize(combined_portfolio_value)
normalized_indexes = {
    'S&P 500': normalize(index_data['^GSPC']),
    'Dow Jones': normalize(index_data['^DJI']),
    'Nasdaq': normalize(index_data['^IXIC'])
}

plt.figure(figsize=(10, 6))

# Plot Portfolio 1 (Monte Carlo Simulation)
plt.plot(normalize(combined_portfolio_value).index, normalize(combined_portfolio_value), label='Portfolio 1 (Monte Carlo)', color='b')

# Plot Portfolio 2 (Combined Professional Optimizations)
plt.plot(normalize(optimized_portfolio_values['Combined Professional']).index, normalize(optimized_portfolio_values['Combined Professional']), 
         label='Portfolio 2 (Combined Professional)', color='g')

# Plot each major index
for name, index_values in normalized_indexes.items():
    plt.plot(index_values.index, normalize(index_values), label=name)

# Add labels and title
plt.title('Portfolio Comparison (1, 2, 3) vs Major Indexes')
plt.xlabel('Date')
plt.ylabel('Normalized Value (Starting at 1)')
plt.legend(loc='upper left')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from scipy.optimize import minimize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from prophet import Prophet

end_date = datetime.today()
one_year_ago = end_date - timedelta(days=2 * 365)
backtest_data = data  # Assuming 'data' contains stock price data

# Step 2: Define the portfolio value calculation function
def portfolio_value(weights, stock_data, initial_investment):
    investment_per_stock = initial_investment * weights
    shares = investment_per_stock / stock_data.iloc[0]
    portfolio_value = (stock_data * shares).sum(axis=1)
    return portfolio_value

# RandomForestRegressor Method with Cross-Validation
def prepare_ml_data(returns):
    data = returns.shift(1).dropna()
    target = returns.loc[data.index]  # Ensure target aligns with the shifted data
    return data, target

X, y = prepare_ml_data(returns)

# Use TimeSeriesSplit for Cross-Validation to avoid look-ahead bias
tscv = TimeSeriesSplit(n_splits=5)
rf_model = RandomForestRegressor(n_estimators=5000, random_state=42)

# Perform cross-validation and train the model
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    rf_model.fit(X_train, y_train)

predicted_returns = rf_model.predict(returns)
predicted_returns_df = pd.DataFrame(predicted_returns, index=returns.index, columns=returns.columns)

# Covariance matrix for portfolio risk
cov_matrix = returns.cov()

# Updated optimization function using Sharpe ratio
def optimized_portfolio_using_predictions(predicted_returns):
    num_assets = predicted_returns.shape[1]
    args = (predicted_returns.mean(), cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
    bounds = tuple((0, 0.2) for asset in range(num_assets))  # Limit weights to 20% max per stock

    def negative_sharpe_ratio(weights, predicted_returns, cov_matrix):
        portfolio_return = np.sum(weights * predicted_returns) * 252
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
        sharpe_ratio = portfolio_return / portfolio_volatility
        return -sharpe_ratio  # Minimize the negative Sharpe ratio

    result = minimize(negative_sharpe_ratio, num_assets * [1. / num_assets,], args=args,
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

ml_optimized_weights = optimized_portfolio_using_predictions(predicted_returns_df)
optimized_portfolio_values = {}
optimized_portfolio_values['ML Optimized'] = portfolio_value(ml_optimized_weights, backtest_data, initial_investment)

# Step 5: Implement Portfolio 4 (LSTM)
def prepare_lstm_data(returns):
    data = returns.values
    X, y = [], []
    for i in range(len(data) - 60):
        X.append(data[i:i+60])  # 60 days of lookback
        y.append(data[i + 60])  # Predict the next day
    return np.array(X), np.array(y)

X_lstm, y_lstm = prepare_lstm_data(returns)

# Define the LSTM model with reduced lookback and dropout
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(X_lstm.shape[1], X_lstm.shape[2])))
lstm_model.add(LSTM(50, activation='relu'))
lstm_model.add(Dense(y_lstm.shape[1]))  # Output layer with same number of assets as input
lstm_model.compile(optimizer='adam', loss='mse')

# Train the LSTM model
lstm_model.fit(X_lstm, y_lstm, epochs=50, batch_size=32)

# Predict using the LSTM model
predicted_lstm_returns = lstm_model.predict(X_lstm[-1].reshape(1, X_lstm.shape[1], X_lstm.shape[2]))[0]

# LSTM optimization with Sharpe ratio
def optimized_portfolio_using_lstm(predicted_returns):
    num_assets = predicted_returns.shape[0]
    args = (predicted_returns, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
    bounds = tuple((0, 0.2) for asset in range(num_assets))

    def negative_sharpe_ratio(weights, predicted_returns, cov_matrix):
        portfolio_return = np.sum(weights * predicted_returns) * 252
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
        sharpe_ratio = portfolio_return / portfolio_volatility
        return -sharpe_ratio

    result = minimize(negative_sharpe_ratio, num_assets * [1. / num_assets,], args=args,
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

lstm_optimized_weights = optimized_portfolio_using_lstm(predicted_lstm_returns)
optimized_portfolio_values['LSTM Optimized'] = portfolio_value(lstm_optimized_weights, backtest_data, initial_investment)

# Step 6: Implement Portfolio 5 (Prophet)
def prepare_prophet_data(returns):
    df = pd.DataFrame({
        'ds': returns.index,
        'y': returns.mean(axis=1)  # Use the average return for prediction
    }).reset_index(drop=True)
    df['ds'] = df['ds'].dt.tz_localize(None)  # Remove timezone information from 'ds' column
    return df

prophet_data = prepare_prophet_data(returns)

prophet_model = Prophet()
prophet_model.fit(prophet_data)

# Forecast for the next 360 periods
future = prophet_model.make_future_dataframe(periods=360)
forecast = prophet_model.predict(future)

# Prophet optimization using Sharpe ratio
def optimized_portfolio_using_prophet(forecasted_returns):
    num_assets = returns.shape[1]  # Number of assets in your portfolio
    args = (forecasted_returns, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
    bounds = tuple((0, 0.2) for _ in range(num_assets))
    
    expected_returns = np.repeat(forecasted_returns['yhat'].values[-360:].mean(), num_assets)

    def negative_sharpe_ratio(weights, expected_returns, cov_matrix):
        portfolio_return = np.sum(weights * expected_returns) * 252
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
        sharpe_ratio = portfolio_return / portfolio_volatility
        return -sharpe_ratio

    result = minimize(negative_sharpe_ratio, num_assets * [1. / num_assets,], args=(expected_returns, cov_matrix),
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

prophet_optimized_weights = optimized_portfolio_using_prophet(forecast)
optimized_portfolio_values['Prophet Optimized'] = portfolio_value(prophet_optimized_weights, backtest_data, initial_investment)

# Step 7: Fetch major indexes for comparison
index_symbols = ['^GSPC', '^DJI', '^IXIC']
index_data = yf.download(index_symbols, start=one_year_ago, end=end_date)['Adj Close']

# Step 8: Plot the comparison of all portfolios and major indexes
def normalize(values):
    return values / values.iloc[0]

normalized_combined_portfolio = normalize(optimized_portfolio_values['ML Optimized'])
normalized_indexes = {
    'S&P 500': normalize(index_data['^GSPC']),
    'Dow Jones': normalize(index_data['^DJI']),
    'Nasdaq': normalize(index_data['^IXIC'])
}

plt.figure(figsize=(10, 6))

# Plot the portfolios
plt.plot(normalized_combined_portfolio.index, normalized_combined_portfolio, label='ML Optimized', color='r')
plt.plot(normalize(optimized_portfolio_values['LSTM Optimized']).index, normalize(optimized_portfolio_values['LSTM Optimized']), 
         label='LSTM Optimized', color='m')
plt.plot(normalize(optimized_portfolio_values['Prophet Optimized']).index, normalize(optimized_portfolio_values['Prophet Optimized']), 
         label='Prophet Optimized', color='c')

# Plot each major index
for name, index_values in normalized_indexes.items():
    plt.plot(index_values.index, index_values, label=name)

# Add labels and title
plt.title('Portfolio Comparison (ML, LSTM, Prophet) vs Major Indexes')
plt.xlabel('Date')
plt.ylabel('Normalized Value (Starting at 1)')
plt.legend(loc='upper left')
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from scipy.optimize import minimize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from prophet import Prophet

# Set date range for backtesting
end_date = datetime.today()
one_year_ago = end_date - timedelta(days=2 * 365)
backtest_data = data  # Assuming 'data' contains stock price data

# Portfolio value calculation function
def portfolio_value(weights, stock_data, initial_investment):
    investment_per_stock = initial_investment * weights
    shares = investment_per_stock / stock_data.iloc[0]
    portfolio_value = (stock_data * shares).sum(axis=1)
    return portfolio_value

# RandomForestRegressor Method with Cross-Validation
def prepare_ml_data(returns):
    data = returns.shift(1).dropna()
    target = returns.loc[data.index]  # Ensure target aligns with the shifted data
    return data, target

X, y = prepare_ml_data(returns)

# TimeSeriesSplit for Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)
rf_model = RandomForestRegressor(n_estimators=5000, random_state=42)

# Cross-validation and train the model
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    rf_model.fit(X_train, y_train)

predicted_returns = rf_model.predict(returns)
predicted_returns_df = pd.DataFrame(predicted_returns, index=returns.index, columns=returns.columns)

# Covariance matrix for portfolio risk
cov_matrix = returns.cov()

# Optimization using Sharpe ratio
def optimized_portfolio_using_predictions(predicted_returns):
    num_assets = predicted_returns.shape[1]
    args = (predicted_returns.mean(), cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
    bounds = tuple((0, 0.2) for asset in range(num_assets))

    def negative_sharpe_ratio(weights, predicted_returns, cov_matrix):
        portfolio_return = np.sum(weights * predicted_returns) * 252
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
        sharpe_ratio = portfolio_return / portfolio_volatility
        return -sharpe_ratio

    result = minimize(negative_sharpe_ratio, num_assets * [1. / num_assets,], args=args,
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

ml_optimized_weights = optimized_portfolio_using_predictions(predicted_returns_df)
optimized_portfolio_values = {}
optimized_portfolio_values['ML Optimized'] = portfolio_value(ml_optimized_weights, backtest_data, initial_investment)

# LSTM Portfolio
def prepare_lstm_data(returns):
    data = returns.values
    X, y = [], []
    for i in range(len(data) - 60):
        X.append(data[i:i+60])  # 60 days of lookback
        y.append(data[i + 60])  # Predict the next day
    return np.array(X), np.array(y)

X_lstm, y_lstm = prepare_lstm_data(returns)

# Define the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(X_lstm.shape[1], X_lstm.shape[2])))
lstm_model.add(LSTM(50, activation='relu'))
lstm_model.add(Dense(y_lstm.shape[1]))  # Output layer with same number of assets as input
lstm_model.compile(optimizer='adam', loss='mse')

# Train the LSTM model
lstm_model.fit(X_lstm, y_lstm, epochs=50, batch_size=32)

# Predict using the LSTM model
predicted_lstm_returns = lstm_model.predict(X_lstm[-1].reshape(1, X_lstm.shape[1], X_lstm.shape[2]))[0]

# Ensure predicted_lstm_returns is a 2D array with shape (1, num_assets)
predicted_lstm_returns = np.reshape(predicted_lstm_returns, (1, -1))

# Now pass the reshaped returns to the optimization function
lstm_optimized_weights = optimized_portfolio_using_predictions(pd.DataFrame(predicted_lstm_returns))
optimized_portfolio_values['LSTM Optimized'] = portfolio_value(lstm_optimized_weights, backtest_data, initial_investment)


# Prophet Portfolio
def prepare_prophet_data(returns):
    df = pd.DataFrame({
        'ds': returns.index,
        'y': returns.mean(axis=1)  # Use the average return for prediction
    }).reset_index(drop=True)
    df['ds'] = df['ds'].dt.tz_localize(None)
    return df

prophet_data = prepare_prophet_data(returns)

prophet_model = Prophet()
prophet_model.fit(prophet_data)

# Forecast for the next 360 periods
future = prophet_model.make_future_dataframe(periods=360)
forecast = prophet_model.predict(future)

# Prophet optimization using Sharpe ratio
def optimized_portfolio_using_prophet(forecasted_returns):
    num_assets = returns.shape[1]  # Number of assets in your portfolio
    expected_returns = np.repeat(forecasted_returns['yhat'].values[-360:].mean(), num_assets)
    
    # Define bounds to limit the weight of each asset
    bounds = tuple((0, 0.2) for _ in range(num_assets))  # Limiting weights to max 20% per asset

    def negative_sharpe_ratio(weights, expected_returns, cov_matrix):
        portfolio_return = np.sum(weights * expected_returns) * 252
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
        sharpe_ratio = portfolio_return / portfolio_volatility
        return -sharpe_ratio

    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})  # Sum of weights must equal 1

    result = minimize(negative_sharpe_ratio, num_assets * [1. / num_assets,], args=(expected_returns, cov_matrix),
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

# Use this updated function
prophet_optimized_weights = optimized_portfolio_using_prophet(forecast)
optimized_portfolio_values['Prophet Optimized'] = portfolio_value(prophet_optimized_weights, backtest_data, initial_investment)

# Modify Monte Carlo Simulation with combined weights to allow 0 weight
weights_combined = {
    'Max Return': 0.20,
    'Min Variance': 0.30,
    'Max Sharpe Ratio': 0.20,
    'Max Treynor Ratio': 0.15,
    'Max Jensen Alpha': 0.15
}

# Generate portfolios using np.random.uniform to allow 0 weights and normalize
portfolios = {
    'Max Return': np.random.uniform(0, 1, len(stocklist_mag)),
    'Min Variance': np.random.uniform(0, 1, len(stocklist_mag)),
    'Max Sharpe Ratio': np.random.uniform(0, 1, len(stocklist_mag)),
    'Max Treynor Ratio': np.random.uniform(0, 1, len(stocklist_mag)),
    'Max Jensen Alpha': np.random.uniform(0, 1, len(stocklist_mag))
}

# Normalize weights to sum to 1
for name in portfolios:
    portfolios[name] /= np.sum(portfolios[name])

# Combine the weights
combined_weights = np.zeros(len(stocklist_mag))
for name, weight in weights_combined.items():
    combined_weights += portfolios[name] * weight
combined_weights /= np.sum(combined_weights)

combined_portfolio_value = portfolio_value(combined_weights, backtest_data, initial_investment)


# Equal weighting of three professional optimizations
mean_variance_weights = min_variance_portfolio(returns)
risk_parity_weights = risk_parity_portfolio(returns)
max_sharpe_weights = max_sharpe_ratio_portfolio(returns, daily_risk_free_rate)

combined_optimized_weights = (mean_variance_weights + risk_parity_weights + max_sharpe_weights) / 3
optimized_portfolio_values['Combined Professional'] = portfolio_value(combined_optimized_weights, backtest_data, initial_investment)

# Fetch major indexes for comparison
index_symbols = ['^GSPC', '^DJI', '^IXIC']
index_data = yf.download(index_symbols, start=one_year_ago, end=end_date)['Adj Close']

# Plot comparison of all portfolios and major indexes
def normalize(values):
    return values / values.iloc[0]

normalized_combined_portfolio = normalize(optimized_portfolio_values['ML Optimized'])
normalized_indexes = {
    'S&P 500': normalize(index_data['^GSPC']),
    'Dow Jones': normalize(index_data['^DJI']),
    'Nasdaq': normalize(index_data['^IXIC'])
}

plt.figure(figsize=(10, 6))

# Plot portfolios
plt.plot(normalized_combined_portfolio.index, normalized_combined_portfolio, label='ML Optimized', color='r')
plt.plot(normalize(optimized_portfolio_values['LSTM Optimized']).index, normalize(optimized_portfolio_values['LSTM Optimized']), label='LSTM Optimized', color='m')
plt.plot(normalize(optimized_portfolio_values['Prophet Optimized']).index, normalize(optimized_portfolio_values['Prophet Optimized']), label='Prophet Optimized', color='c')
plt.plot(normalize(combined_portfolio_value).index, normalize(combined_portfolio_value), label='Portfolio 1 (Monte Carlo)', color='b')
plt.plot(normalize(optimized_portfolio_values['Combined Professional']).index, normalize(optimized_portfolio_values['Combined Professional']), label='Portfolio 2 (Combined Professional)', color='g')

# Plot each major index
for name, index_values in normalized_indexes.items():
    plt.plot(index_values.index, index_values, label=name)

# Add labels and title
plt.title('Portfolio Comparison vs Major Indexes')
plt.xlabel('Date')
plt.ylabel('Normalized Value (Starting at 1)')
plt.legend(loc='upper left')
plt.grid(True)
plt.show()


In [ ]:
# Step 1: Check optimized weights for each strategy
optimized_weights = {
    'ML Optimized': ml_optimized_weights,
    'LSTM Optimized': lstm_optimized_weights,
    'Prophet Optimized': prophet_optimized_weights,
    'Monte Carlo (Combined Weights)': combined_weights,
    'Combined Professional Optimizations': combined_optimized_weights
}

# Step 2: Function to check if zero weights are allowed
def check_zero_weights(weights, strategy_name):
    zero_weights = np.isclose(weights, 0)  # Check if any weights are approximately zero
    if np.any(zero_weights):
        print(f"{strategy_name} allows zero weights.")
    else:
        print(f"{strategy_name} does not allow zero weights.")

# Step 3: Check for zero weights in each strategy
for strategy, weights in optimized_weights.items():
    check_zero_weights(weights, strategy)


# show the stats

In [ ]:
# Display the portfolio weightings for the 5 different portfolios
def display_portfolio_weights(portfolio_name, weights, stock_names):
    weights_df = pd.DataFrame({
        'Stock': stock_names,
        'Weight': weights
    })
    weights_df['Weight'] = weights_df['Weight'].apply(lambda x: f"{x:.4f}")
    print(f"\nPortfolio Weights - {portfolio_name}:\n", weights_df)

# Assuming `stocklist_mag` contains the list of stock names
stock_names = stocklist_mag  # Replace with the correct list of stock names

# Portfolio 1: Monte Carlo Simulation (Combined Weights)
display_portfolio_weights("Monte Carlo Simulation (Combined)", combined_weights, stock_names)

# Portfolio 2: Combined Professional Optimizations
display_portfolio_weights("Combined Professional Optimizations", combined_optimized_weights, stock_names)

# Portfolio 3: Machine Learning Optimization
display_portfolio_weights("Machine Learning Optimization", ml_optimized_weights, stock_names)

# Portfolio 4: LSTM Optimization
display_portfolio_weights("LSTM Optimization", lstm_optimized_weights, stock_names)

# Portfolio 5: Prophet Optimization
display_portfolio_weights("Prophet Optimization", prophet_optimized_weights, stock_names)



In [ ]:
# Step 8: Calculate major statistics for each portfolio and shares purchased

# Function to calculate portfolio statistics
def calculate_portfolio_stats(portfolio_values, returns, weights, stock_data, initial_investment, risk_free_rate):
    portfolio_returns = portfolio_values.pct_change().dropna()

    # Annualized Return
    annualized_return = np.mean(portfolio_returns) * 252

    # Annualized Volatility
    annualized_volatility = np.std(portfolio_returns) * np.sqrt(252)

    # Sharpe Ratio
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility

    # Calculate shares purchased
    investment_per_stock = initial_investment * weights
    shares_purchased = investment_per_stock / stock_data.iloc[0]

    return {
        "Annualized Return": annualized_return,
        "Annualized Volatility": annualized_volatility,
        "Sharpe Ratio": sharpe_ratio,
        "Shares Purchased": shares_purchased
    }

# Print statistics and shares purchased for each portfolio

portfolio_stats = {}
portfolios_to_check = {
    "Portfolio 1 (Monte Carlo)": combined_portfolio_value,
    "Portfolio 2 (Combined Professional)": optimized_portfolio_values['Combined Professional'],
    "Portfolio 3 (Machine Learning)": optimized_portfolio_values['ML Optimized'],
    "Portfolio 4 (LSTM)": optimized_portfolio_values['LSTM Optimized'],
    "portfolio 5 (Prophet)": optimized_portfolio_values['Prophet Optimized']
}

for portfolio_name, portfolio_value in portfolios_to_check.items():
    stats = calculate_portfolio_stats(portfolio_value, returns, combined_weights, backtest_data, initial_investment, daily_risk_free_rate)
    portfolio_stats[portfolio_name] = stats
    
    # Print statistics
    print(f"\n{portfolio_name}:")
    print(f"Annualized Return: {stats['Annualized Return']:.2%}")
    print(f"Annualized Volatility: {stats['Annualized Volatility']:.2%}")
    print(f"Sharpe Ratio: {stats['Sharpe Ratio']:.2f}")
    
    # Print shares purchased
    print("Shares Purchased per Stock:")
    for stock, shares in zip(stocklist_mag, stats['Shares Purchased']):
        print(f"{stock}: {shares:.4f} shares")


In [ ]:
def calculate_portfolio_stats(portfolio_values, returns, weights, stock_data, initial_investment, risk_free_rate, market_returns):
    portfolio_returns = portfolio_values.pct_change().dropna()

    # Annualized Return
    annualized_return = np.mean(portfolio_returns) * 252

    # Annualized Volatility
    annualized_volatility = np.std(portfolio_returns) * np.sqrt(252)

    # Sharpe Ratio
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility

    # Sortino Ratio
    downside_risk = np.std(portfolio_returns[portfolio_returns < 0]) * np.sqrt(252)
    sortino_ratio = (annualized_return - risk_free_rate) / downside_risk if downside_risk != 0 else np.nan

    # Maximum Drawdown
    rolling_max = portfolio_values.cummax()
    drawdown = (portfolio_values - rolling_max) / rolling_max
    max_drawdown = drawdown.min()

    # Calmar Ratio
    calmar_ratio = annualized_return / abs(max_drawdown) if max_drawdown != 0 else np.nan

    # Beta (Sensitivity to the market return)
    covariance = np.cov(portfolio_returns, market_returns)[0, 1]
    beta = covariance / np.var(market_returns)

    # Treynor Ratio
    treynor_ratio = (annualized_return - risk_free_rate) / beta if beta != 0 else np.nan

    # Calculate shares purchased
    investment_per_stock = initial_investment * weights
    shares_purchased = investment_per_stock / stock_data

    # Calculate the value of each stock in the portfolio (current price * shares purchased)
    stock_values = shares_purchased * stock_data

    # Calculate the percentage of the portfolio value represented by each stock
    total_portfolio_value = stock_values.sum()
    stock_percentages = (stock_values / total_portfolio_value) * 100

    return {
        "Annualized Return": annualized_return,
        "Annualized Volatility": annualized_volatility,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Maximum Drawdown": max_drawdown,
        "Calmar Ratio": calmar_ratio,
        "Beta": beta,
        "Treynor Ratio": treynor_ratio,
        "Shares Purchased": shares_purchased,
        "Stock Values": stock_values,
        "Stock Percentages": stock_percentages
    }

# Assuming market_return is defined (e.g., from S&P 500 or another market index)
for portfolio_name, portfolio_value in portfolios_to_check.items():
    stats = calculate_portfolio_stats(portfolio_value, returns, combined_weights, latest_stock_data, initial_investment, daily_risk_free_rate, market_returns)
    portfolio_stats[portfolio_name] = stats

    # Print statistics
    print(f"\n{portfolio_name}:")
    print(f"Annualized Return: {stats['Annualized Return']:.2%}")
    print(f"Annualized Volatility: {stats['Annualized Volatility']:.2%}")
    print(f"Sharpe Ratio: {stats['Sharpe Ratio']:.2f}")
    print(f"Sortino Ratio: {stats['Sortino Ratio']:.2f}")
    print(f"Maximum Drawdown: {stats['Maximum Drawdown']:.2%}")
    print(f"Calmar Ratio: {stats['Calmar Ratio']:.2f}")
    print(f"Beta: {stats['Beta']:.2f}")
    print(f"Treynor Ratio: {stats['Treynor Ratio']:.2f}")

    # Print shares purchased and their value
    print("Shares Purchased and Value per Stock:")
    for stock, shares, value, percentage in zip(stocklist_mag, stats['Shares Purchased'], stats['Stock Values'], stats['Stock Percentages']):
        print(f"{stock}: {shares:.4f} shares, Value: ${value:.2f}, Percentage of Portfolio: {percentage:.2f}%")


# prediction method 1 and 2

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.linear_model import LinearRegression
import zipfile
import io
import requests
import pandas as pd
import requests
import zipfile
import io

# Step 1: Download the Fama-French data
url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip"
response = requests.get(url)
zipped_file = zipfile.ZipFile(io.BytesIO(response.content))

# Read the Fama-French CSV and skip the non-data rows
with zipped_file.open('F-F_Research_Data_Factors.CSV') as ff_file:
    fama_french_data = pd.read_csv(ff_file, skiprows=3)

# Step 2: Clean the data by removing the footer and extra rows
# Find the row with the first instance of "Annual Factors" (which marks the start of non-data rows)
stop_index = fama_french_data[fama_french_data.iloc[:, 0].str.contains("Annual Factors", na=False)].index[0]
fama_french_data = fama_french_data[:stop_index]  # Only keep data rows before this index

# Rename columns to appropriate labels
fama_french_data.columns = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RF']

# Ensure the Date column is correctly formatted (skip rows with any non-date value)
fama_french_data['Date'] = pd.to_datetime(fama_french_data['Date'], format='%Y%m', errors='coerce').dropna()

# Set the date as index
fama_french_data = fama_french_data.set_index('Date')

# Convert percentage values to decimals for easier calculation
fama_french_data = fama_french_data.astype(float) / 100  # Convert percentages to decimals

# Display the first few rows to ensure the cleaning is correct
print(fama_french_data.head())



end_date = datetime.today()  # Keep this as a datetime object
one_year_ago = end_date - timedelta(days=365)
# Filter the data to match our stock period
start_date_str = one_year_ago.strftime('%Y-%m')  # Convert the datetime object to string format
end_date_str = end_date.strftime('%Y-%m')  # Convert the datetime object to string format
fama_french_filtered = fama_french_data.loc[start_date_str:end_date_str]

# Prepare the market returns (Market - RF) and risk-free rate
market_excess_return = fama_french_filtered['Mkt-RF']
risk_free_rate_fama = fama_french_filtered['RF']
SMB = fama_french_filtered['SMB']
HML = fama_french_filtered['HML']

risk_free_rate_fama = risk_free_rate_fama.tz_localize(None)


# Step 2: Calculate the excess returns of the portfolios
portfolios_excess_returns = {}
for portfolio_name, portfolio_value in portfolios_to_check.items():
    # Calculate excess returns (portfolio returns - risk-free rate)
    portfolio_returns = portfolio_value.pct_change().dropna()
    # Remove timezone information from the index
    portfolio_returns.index = portfolio_returns.index.tz_localize(None)

    portfolio_excess_return = portfolio_returns - risk_free_rate_fama.reindex(portfolio_returns.index, method='ffill')
    portfolios_excess_returns[portfolio_name] = portfolio_excess_return

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Step 3: CAPM Regression (for each portfolio)
def capm_prediction(excess_returns, market_excess_return, risk_free_rate_fama):
    # Align indices and remove NaNs
    excess_returns, market_excess_return = excess_returns.align(market_excess_return, join='inner')
    
    # Reshape data for regression
    X = market_excess_return.values.reshape(-1, 1)
    y = excess_returns.values
    
    # Remove NaNs (if any) in case of data misalignment
    valid_mask = ~np.isnan(X).flatten() & ~np.isnan(y)
    X = X[valid_mask]
    y = y[valid_mask]
    
    # Perform linear regression
    capm_model = LinearRegression()
    capm_model.fit(X, y)
    
    # Beta coefficient
    beta = capm_model.coef_[0]
    
    # Predict the next 3 months using the average market return
    future_market_return = market_excess_return.mean()  # Using historical average
    expected_return = risk_free_rate_fama.mean() + beta * future_market_return
    return expected_return, beta

capm_predictions = {}
for portfolio_name, excess_returns in portfolios_excess_returns.items():
    expected_return, beta = capm_prediction(excess_returns, market_excess_return, risk_free_rate_fama)
    capm_predictions[portfolio_name] = (expected_return, beta)

# Step 4: Fama-French Regression (for each portfolio)
def fama_french_prediction(excess_returns, market_excess_return, SMB, HML, risk_free_rate_fama):
    # Align indices and remove NaNs
    excess_returns, market_excess_return = excess_returns.align(market_excess_return, join='inner')
    excess_returns, SMB = excess_returns.align(SMB, join='inner')
    excess_returns, HML = excess_returns.align(HML, join='inner')
    
    # Combine the Fama-French factors into a single DataFrame and drop NaNs
    X = pd.DataFrame({
        'Mkt-RF': market_excess_return,
        'SMB': SMB,
        'HML': HML
    }).dropna()
    y = excess_returns.loc[X.index].values
    
    # Perform multivariate linear regression
    fama_french_model = LinearRegression()
    fama_french_model.fit(X, y)
    
    # Coefficients (betas)
    beta_market = fama_french_model.coef_[0]
    beta_smb = fama_french_model.coef_[1]
    beta_hml = fama_french_model.coef_[2]
    
    # Predict the next 3 months using the average factors
    future_market_return = market_excess_return.mean()  # Using historical average
    future_SMB = SMB.mean()
    future_HML = HML.mean()
    
    expected_return = risk_free_rate_fama.mean() + beta_market * future_market_return + beta_smb * future_SMB + beta_hml * future_HML
    return expected_return, (beta_market, beta_smb, beta_hml)

fama_french_predictions = {}
for portfolio_name, excess_returns in portfolios_excess_returns.items():
    expected_return, betas = fama_french_prediction(excess_returns, market_excess_return, SMB, HML, risk_free_rate_fama)
    fama_french_predictions[portfolio_name] = (expected_return, betas)

# Step 5: Print the predictions for the next 3 months
print("CAPM Predictions (Next 3 months):")
for portfolio_name, (expected_return, beta) in capm_predictions.items():
    print(f"{portfolio_name}: Expected Return: {expected_return:.4%}, Beta: {beta:.4f}")

print("\nFama-French Predictions (Next 3 months):")
for portfolio_name, (expected_return, betas) in fama_french_predictions.items():
    beta_market, beta_smb, beta_hml = betas
    print(f"{portfolio_name}: Expected Return: {expected_return:.4%}, Beta_Market: {beta_market:.4f}, Beta_SMB: {beta_smb:.4f}, Beta_HML: {beta_hml:.4f}")
